### Calculate installed capacity at offshore area vs max possible

In [107]:
import pandas as pd
from constants import WIND_FARM_NODES

pd.options.mode.chained_assignment = None  # default='warn'

SPECIAL_AREAS = ["SørvestA", "SørvestD", "SørvestE"] # Areas where both techs are possible but both can't surpass area capacity

#CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
CASES = ["Base", "ConstTest"]
DF_BY_CASES_SPLIT = dict({})
DF_BY_CASES_COMBO = dict({})

GEN_TYPE_MAPPING = dict({
    "Wind_offshr_grounded": "Windoffshoregrounded",
    "Wind_offshr_floating": "Windoffshorefloating",
})

max_capacaities = pd.read_csv("../EMPIRE_extension/Output/offshore_capacities_base.csv")\
                        .rename(columns={"GeneratorTechnology": "GeneratorType", "generatorMaxInstallCapacity in MW": "Max capacity [MW]"})
max_capacaities["Node"] = max_capacaities["Node"].apply(lambda n: n.replace(" ", ""))
max_capacaities["GeneratorType"] = max_capacaities["GeneratorType"].apply(lambda g: GEN_TYPE_MAPPING[g])
max_capacaities = max_capacaities.drop(columns=["area_km2"])
max_capacaities

Node         GeneratorType  Max capacity [MW]
0         MorayFirth  Windoffshoregrounded        5465.000000
1       FirthofForth  Windoffshoregrounded       10530.000000
2         DoggerBank  Windoffshoregrounded       16260.000000
3            Hornsea  Windoffshoregrounded       12055.000000
4       OuterDowsing  Windoffshoregrounded        6265.000000
5            Norfolk  Windoffshoregrounded        8130.000000
6         EastAnglia  Windoffshoregrounded        6755.000000
7           Borssele  Windoffshoregrounded        4881.916667
8     HollandseeKust  Windoffshoregrounded        6664.166667
9   HelgoländerBucht  Windoffshoregrounded       22910.625000
10          Nordsøen  Windoffshoregrounded       20110.000000
11        MorayFirth  Windoffshorefloating       11645.000000
12      FirthofForth  Windoffshorefloating       27745.000000
13         NordvestA  Windoffshorefloating       56535.000000
14         NordvestC  Windoffshorefloating       27910.000000
15        VestavindA  Windoffshorefloating        9420.000000
16        SønnavindA  Windoffshorefloating       14500.000000
17          SørvestC  Windoffshoregrounded        8830.000000
18         NordvestB  Windoffshorefloating       17185.000000
19        VestavindF  Windoffshorefloating        9945.000000
20        VestavindB  Windoffshorefloating       14925.000000
21        VestavindC  Windoffshorefloating        5320.000000
22        VestavindD  Windoffshorefloating        3670.000000
23          SørvestF  Windoffshoregrounded       13510.000000
24          SørvestB  Windoffshoregrounded       10895.000000
25        NordavindB  Windoffshorefloating       11195.000000
26        NordavindA  Windoffshorefloating       21375.000000
27        NordavindD  Windoffshorefloating       20560.000000
28        NordavindC  Windoffshorefloating        5270.000000
29        VestavindE  Windoffshorefloating        7375.000000
30          SørvestE  Windoffshorefloating        5080.000000
31          SørvestA  Windoffshorefloating        7280.000000
32          SørvestD  Windoffshorefloating        6075.000000
33          SørvestE  Windoffshoregrounded        5080.000000
34          SørvestA  Windoffshoregrounded        7280.000000
35          SørvestD  Windoffshoregrounded        6075.000000

In [116]:
max_capacaities_combo = max_capacaities.copy()
max_capacaities_combo["Max capacity [MW]"] = max_capacaities_combo.groupby(["Node"])["Max capacity [MW]"].transform("sum")
max_capacaities_combo["Max capacity [MW]"] = max_capacaities_combo.apply(lambda row: row["Max capacity [MW]"] / 2 if row["Node"]\
                                                                            in SPECIAL_AREAS else row["Max capacity [MW]"], axis=1)
max_capacaities_combo = max_capacaities_combo.drop_duplicates("Node").drop(columns=["GeneratorType"]).reset_index(drop=True)
max_capacaities_combo

Node  Max capacity [MW]
0         MorayFirth       17110.000000
1       FirthofForth       38275.000000
2         DoggerBank       16260.000000
3            Hornsea       12055.000000
4       OuterDowsing        6265.000000
5            Norfolk        8130.000000
6         EastAnglia        6755.000000
7           Borssele        4881.916667
8     HollandseeKust        6664.166667
9   HelgoländerBucht       22910.625000
10          Nordsøen       20110.000000
11         NordvestA       56535.000000
12         NordvestC       27910.000000
13        VestavindA        9420.000000
14        SønnavindA       14500.000000
15          SørvestC        8830.000000
16         NordvestB       17185.000000
17        VestavindF        9945.000000
18        VestavindB       14925.000000
19        VestavindC        5320.000000
20        VestavindD        3670.000000
21          SørvestF       13510.000000
22          SørvestB       10895.000000
23        NordavindB       11195.000000
24        NordavindA       21375.000000
25        NordavindD       20560.000000
26        NordavindC        5270.000000
27        VestavindE        7375.000000
28          SørvestE        5080.000000
29          SørvestA        7280.000000
30          SørvestD        6075.000000

In [117]:
for case in CASES:
    _df = pd.read_csv(f'Results/{case}/results_output_gen.csv')

    # Offshore nodes
    df_wind_farm_nodes_combo = _df[_df['Node'].isin(WIND_FARM_NODES)]
    df_wind_farm_nodes_split= _df[_df['Node'].isin(WIND_FARM_NODES)]

    # Sum floating + grounded cap (combo)
    df_wind_farm_nodes_combo["genInstalledCap_MW"] = df_wind_farm_nodes_combo.groupby(['Node', 'Period'])['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.drop_duplicates(("Node", "Period"), ignore_index=True)\
                                [["Node", "Period", "genInstalledCap_MW"]]
    df_wind_farm_nodes_combo = df_wind_farm_nodes_combo.merge(max_capacaities_combo, on=["Node"], how="left")
    df_wind_farm_nodes_combo["Installed capacity of max (%)"] = round(df_wind_farm_nodes_combo["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_combo["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_COMBO[case] = df_wind_farm_nodes_combo

    # Floating + grounded splitted
    df_wind_farm_nodes_split["genInstalledCap_MW"] = df_wind_farm_nodes_split.groupby(['Node', 'Period', 'GeneratorType'])\
                                ['genInstalledCap_MW'].transform('sum')
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.drop_duplicates(("Node", "Period", 'GeneratorType'), ignore_index=True)\
                                [["Node", "Period", 'GeneratorType', "genInstalledCap_MW"]]
    df_wind_farm_nodes_split = df_wind_farm_nodes_split.merge(max_capacaities, on=["Node", 'GeneratorType'], how="left")
    df_wind_farm_nodes_split["Installed capacity of max (%)"] = round(df_wind_farm_nodes_split["genInstalledCap_MW"] \
                                                                        / df_wind_farm_nodes_split["Max capacity [MW]"] * 100, 1)
    DF_BY_CASES_SPLIT[case] = df_wind_farm_nodes_split

In [123]:
FILTER_CASE = "ConstTest"
FILTER_PERIOD = "2045-2050"
COMBO = False # If False "split" by floating/grounded; else total cap in area

filtered_df = DF_BY_CASES_COMBO[FILTER_CASE] if COMBO else DF_BY_CASES_SPLIT[FILTER_CASE]
filtered_df = filtered_df[(filtered_df["Period"] == FILTER_PERIOD) & filtered_df["Max capacity [MW]"] > 0]
filtered_df["genInstalledCap_MW"] = round(filtered_df["genInstalledCap_MW"])
filtered_df["Max capacity [MW]"] = round(filtered_df["Max capacity [MW]"])
filtered_df = filtered_df.sort_values(by="genInstalledCap_MW", ascending=False).reset_index(drop=True)
filtered_df

Node     Period         GeneratorType  genInstalledCap_MW  \
0   HelgoländerBucht  2045-2050  Windoffshoregrounded             22911.0   
1           Nordsøen  2045-2050  Windoffshoregrounded             20110.0   
2         DoggerBank  2045-2050  Windoffshoregrounded             16260.0   
3           SørvestF  2045-2050  Windoffshoregrounded             13510.0   
4            Hornsea  2045-2050  Windoffshoregrounded             12055.0   
5         SønnavindA  2045-2050  Windoffshorefloating             11034.0   
6           SørvestB  2045-2050  Windoffshoregrounded             10895.0   
7       FirthofForth  2045-2050  Windoffshoregrounded             10530.0   
8          NordvestC  2045-2050  Windoffshorefloating              9836.0   
9         VestavindA  2045-2050  Windoffshorefloating              9420.0   
10          SørvestC  2045-2050  Windoffshoregrounded              8830.0   
11           Norfolk  2045-2050  Windoffshoregrounded              8130.0   
12          SørvestA  2045-2050  Windoffshoregrounded              7280.0   
13        EastAnglia  2045-2050  Windoffshoregrounded              6755.0   
14    HollandseeKust  2045-2050  Windoffshoregrounded              6664.0   
15      OuterDowsing  2045-2050  Windoffshoregrounded              6265.0   
16          SørvestD  2045-2050  Windoffshoregrounded              6075.0   
17        MorayFirth  2045-2050  Windoffshoregrounded              5465.0   
18         NordvestB  2045-2050  Windoffshorefloating              5367.0   
19          SørvestE  2045-2050  Windoffshoregrounded              5080.0   
20         NordvestA  2045-2050  Windoffshorefloating              5000.0   
21          Borssele  2045-2050  Windoffshoregrounded              4882.0   
22        VestavindC  2045-2050  Windoffshorefloating              3901.0   
23        MorayFirth  2045-2050  Windoffshorefloating              3509.0   
24        VestavindB  2045-2050  Windoffshorefloating              2422.0   
25        NordavindA  2045-2050  Windoffshorefloating              1082.0   
26        NordavindC  2045-2050  Windoffshorefloating                 0.0   
27          SørvestA  2045-2050  Windoffshorefloating                 0.0   
28          SørvestE  2045-2050  Windoffshorefloating                 0.0   
29        VestavindE  2045-2050  Windoffshorefloating                 0.0   
30      FirthofForth  2045-2050  Windoffshorefloating                 0.0   
31        NordavindD  2045-2050  Windoffshorefloating                 0.0   
32        NordavindB  2045-2050  Windoffshorefloating                 0.0   
33        VestavindD  2045-2050  Windoffshorefloating                 0.0   
34        VestavindF  2045-2050  Windoffshorefloating                 0.0   
35          SørvestD  2045-2050  Windoffshorefloating                 0.0   

    Max capacity [MW]  Installed capacity of max (%)  
0             22911.0                          100.0  
1             20110.0                          100.0  
2             16260.0                          100.0  
3             13510.0                          100.0  
4             12055.0                          100.0  
5             14500.0                           76.1  
6             10895.0                          100.0  
7             10530.0                          100.0  
8             27910.0                           35.2  
9              9420.0                          100.0  
10             8830.0                          100.0  
11             8130.0                          100.0  
12             7280.0                          100.0  
13             6755.0                          100.0  
14             6664.0                          100.0  
15             6265.0                          100.0  
16             6075.0                          100.0  
17             5465.0                          100.0  
18            17185.0                           31.2  
19             5080.0                          100.0  
20         